In [19]:
import numpy as np
from scipy.special import logsumexp

def forward(loga, logb, T, logpi, observations):
    logalpha = np.empty((T, 2))
    logalpha[0, 0] = logpi[0] + logb[0, observations[0]]
    logalpha[0, 1] = logpi[1] + logb[1, observations[0]]
    for t in range(1, T):
        for j in range(2):
            logterms = [loga[i, j] + logalpha[t - 1, i] for i in range(2)]
            logalpha[t, j] = logsumexp(logterms) + logb[j, observations[t]]
        
    return logalpha

def backward(loga, logb, T, observations):
    logbeta = np.empty((T, 2))
    logbeta[T-1, :] = 0
    for t in range(T - 2, -1, -1):
        for i in range(2):
            logterms = [loga[i, j] + logb[j, observations[t+1]] + logbeta[t+1, j] for j in range(2)]
            logbeta[t, i] = logsumexp(logterms)
            
    return logbeta

def compute_gamma(logalpha, logbeta, T):
    loggamma = np.empty((T, 2))
    for t in range(T):
        for i in range(2):
            loggamma[t, i] = logalpha[t, i] + logbeta[t, i] - logsumexp([
                logalpha[t, j] + logbeta[t, j] for j in range(2)
            ])

    return loggamma

def compute_xi(logalpha, logbeta, loga, logb, observations, T):
    xi = np.empty((T, 2, 2))
    for t in range(T - 1):
        for i in range(2):
            for j in range(2):
                logterms = []
                for k in range(2):
                    for l in range(2):
                        logterms.append(
                            logalpha[t, k] + loga[k, l] + logb[l, observations[t + 1]] + logbeta[t + 1, l]
                        )
                xi[t, i, j] = (
                    logalpha[t, i] + loga[i, j] + logb[j, observations[t + 1]] + logbeta[t + 1, j]
                    - logsumexp(logterms)
                )
    return xi

def compute_a(loggamma, logxi, T):
    loga = np.empty((2, 2))
    for i in range(2):
        for j in range(2):
            loga[i, j] = (
                logsumexp([logxi[t, i, j] for t in range(T - 1)])
                - logsumexp([loggamma[t, i] for t in range(T - 1)])
            )
    return loga

def compute_b(loggamma, observations, T):
    logb = np.empty((2, 2))
    for i in range(2):
        for k in range(2):
            logb[i, k] = (
                logsumexp([loggamma[t, i] for t in range(T) if observations[t] == k])
                - logsumexp([loggamma[t, i] for t in range(T)])
            )
    return logb

In [23]:
import numpy as np

T = 2000

# hidden data (that we try to estimate)
A = np.array([[0.5, 0.5], [0.8, 0.2]]) # columns and rows indexed by (rain, no rain)
B = np.array([[0.8, 0.2], [0.15, 0.85]]) # rows indexed by (rain, no rain), columns by (umbrella, no umbrella)
pi = np.array([0.5, 0.5]) # (prob for rain, prob for no rain)

# we use A, B and pi to construct a HMM environment with two states and two observable values
observations = np.empty(T)
states = np.empty(T)

states[0] = int(np.random.uniform() > pi[0]) # 0 = rain
observations[0] = int(np.random.uniform() > B[int(states[0]), 0]) # 0 = umbrella

for t in range(1, T):
    prob_rain = A[int(states[t - 1]), 0]
    states[t] = int(np.random.uniform() > prob_rain)
    prob_umb = B[int(states[t]), 0]
    observations[t] = int(np.random.uniform() > prob_umb)

observations = [int(t) for t in observations]
states = [int(t) for t in states]

A_real = A.copy()
B_real = B.copy()
pi_real = pi.copy()

ll_tol = 0.2
n_attempts = 100

ll_list = []
A_list = []
B_list = []
diff_norms_A = []
diff_norms_B = []

for _ in range(n_attempts):

    prev_ll = 0
    ll = 1.0

    # # initial guesses
    # A = np.empty((2, 2)) # columns and rows indexed by (rain, no rain)
    # A[:, 0] = np.random.uniform(size=2)
    # A[:, 1] = 1 - A[:, 0]
    # B = np.empty((2, 2)) # rows indexed by (rain, no rain), columns by (umbrella, no umbrella)
    # B[:, 0] = np.random.uniform(size=2)
    # B[:, 1] = 1 - B[:, 0]
    # prob_start_rain = np.random.uniform()
    # pi = np.array([prob_start_rain, 1 - prob_start_rain]) # (prob for rain, prob for no rain)

    A_real_first_col = A_real[:, 0]
    A_first_col = (A_real_first_col + np.random.normal(0, 0.2, 2)).clip(0.01, 1)
    A = np.empty((2, 2))
    A[:, 0] = A_first_col
    A[:, 1] = 1 - A_first_col

    B_real_first_col = B_real[:, 0]
    B_first_col = (B_real_first_col + np.random.normal(0, 0.2, 2)).clip(0.01, 1)
    B = np.empty((2, 2))
    B[:, 0] = B_first_col
    B[:, 1] = 1 - B_first_col

    p = (pi_real[0] + np.random.normal(0, 0.2)).clip(0.01, 1)
    pi = np.array([p, 1 - p])
    #pi = pi_real

    print('FIRST GUESSES:')
    print(A)
    print(B)
    print(pi)

    # take logs
    loga = np.log(A)
    logb = np.log(B)
    logpi = np.log(pi)

    while abs(prev_ll - ll) > ll_tol:
        prev_ll = ll

        # E step
        logalpha = forward(loga, logb, T, logpi, observations)
        logbeta = backward(loga, logb, T, observations)
        loggamma = compute_gamma(logalpha, logbeta, T)
        logxi = compute_xi(logalpha, logbeta, loga, logb, observations, T)

        # M step
        loga = compute_a(loggamma, logxi, T)
        logb = compute_b(loggamma, observations, T)
        logpi = loggamma[0, :]

        log_ll = logsumexp([logalpha[-1, i] for i in range(2)])
        print('log_likelyhood:', log_ll)

    

    print('final estimated values:')

    A = np.exp(loga)
    A /= A.sum(axis=1, keepdims=True)
    B = np.exp(logb)
    B /= B.sum(axis=1, keepdims=True)
    pi = np.exp(logpi)
    pi /= pi.sum()

    A = np.clip(A, 1e-10, 1)
    B = np.clip(B, 1e-10, 1)

    if np.isnan(A).any() or np.isnan(B).any():
        continue

    print('A:'); print(A)
    print('B:'); print(B)
    print('pi:'); print(pi)

    print('true values:')
    print('A:'); print(A_real)
    print('B:'); print(B_real)
    print('pi:'); print(pi_real)

    A_list.append(A)
    B_list.append(B)
    ll_list.append(log_ll)

    #diff_norms_A.append(np.linalg.norm(A - A_real))
    #diff_norms_B.append(np.linalg.norm(B - B_real))

idx = np.argmax(ll_list)
print("-------------------")
print('highest_ll:', ll_list[idx], "corresponding with index", idx)
print('corresponding A:', A_list[idx])
print('corresponding B:', B_list[idx])
print()
print('true values:')
print('A:'); print(A_real)
print('B:'); print(B_real) 
print('pi:'); print(pi_real)
print('---------')
# idx = np.argmin(diff_norms_A)
# best_guess_A = A_list[idx]
# print('best guess A:', best_guess_A, "corresponding with index", idx)
# idx = np.argmin(diff_norms_B)
# best_guess_B = B_list[idx]
# print('best guess B:', best_guess_B, "corresponding with index", idx)
# print('---------------------------')
print(ll_list)
print(A_list)
print(B_list)



FIRST GUESSES:
[[0.82778793 0.17221207]
 [0.56564083 0.43435917]]
[[0.93402028 0.06597972]
 [0.01       0.99      ]]
[0.36886944 0.63113056]
log_likelyhood: -1585.2132679312235
final estimated values:
A:
[[0.64170169 0.35829831]
 [0.59726352 0.40273648]]
B:
[[0.86048384 0.13951616]
 [0.00934665 0.99065335]]
pi:
[0.96573411 0.03426589]
true values:
A:
[[0.5 0.5]
 [0.8 0.2]]
B:
[[0.8  0.2 ]
 [0.15 0.85]]
pi:
[0.5 0.5]
FIRST GUESSES:
[[0.84883158 0.15116842]
 [0.62607207 0.37392793]]
[[0.65314198 0.34685802]
 [0.01       0.99      ]]
[0.48159033 0.51840967]
log_likelyhood: -1398.2050187546536
final estimated values:
A:
[[0.85219226 0.14780774]
 [0.70673908 0.29326092]]
B:
[[0.65209601 0.34790399]
 [0.01230398 0.98769602]]
pi:
[0.98040682 0.01959318]
true values:
A:
[[0.5 0.5]
 [0.8 0.2]]
B:
[[0.8  0.2 ]
 [0.15 0.85]]
pi:
[0.5 0.5]
FIRST GUESSES:
[[0.27704679 0.72295321]
 [0.92818819 0.07181181]]
[[0.81345834 0.18654166]
 [0.03992767 0.96007233]]
[0.95260377 0.04739623]
log_likelyhood: -14

C:\Users\tobia\AppData\Local\Temp\ipykernel_10260\3814822106.py:77: RuntimeWarning: divide by zero encountered in log
  logb = np.log(B)


log_likelyhood: -1381.0317143534285
final estimated values:
A:
[[0.42467045 0.57532955]
 [0.54916754 0.45083246]]
B:
[[1.00000000e+00 1.00000000e-10]
 [1.03509871e-01 8.96490129e-01]]
pi:
[0.98496666 0.01503334]
true values:
A:
[[0.5 0.5]
 [0.8 0.2]]
B:
[[0.8  0.2 ]
 [0.15 0.85]]
pi:
[0.5 0.5]
FIRST GUESSES:
[[0.26152092 0.73847908]
 [0.77148181 0.22851819]]
[[1.         0.        ]
 [0.44749548 0.55250452]]
[0.43802651 0.56197349]
log_likelyhood: -1637.4672138477813
final estimated values:
A:
[[0.20247835 0.79752165]
 [0.57802581 0.42197419]]
B:
[[1.00000000e+00 1.00000000e-10]
 [2.09055039e-01 7.90944961e-01]]
pi:
[0.8491424 0.1508576]
true values:
A:
[[0.5 0.5]
 [0.8 0.2]]
B:
[[0.8  0.2 ]
 [0.15 0.85]]
pi:
[0.5 0.5]
FIRST GUESSES:
[[0.8246683  0.1753317 ]
 [0.70676233 0.29323767]]
[[0.78209017 0.21790983]
 [0.01       0.99      ]]
[0.61275496 0.38724504]
log_likelyhood: -1424.4727535621657
final estimated values:
A:
[[0.7693406  0.2306594 ]
 [0.71613479 0.28386521]]
B:
[[0.71321071 

C:\Users\tobia\AppData\Local\Temp\ipykernel_10260\3814822106.py:76: RuntimeWarning: divide by zero encountered in log
  loga = np.log(A)


log_likelyhood: -1378.7765475162273
final estimated values:
A:
[[4.63511804e-01 5.36488196e-01]
 [1.00000000e+00 1.00000000e-10]]
B:
[[0.57422818 0.42577182]
 [0.48051971 0.51948029]]
pi:
[0.39427246 0.60572754]
true values:
A:
[[0.5 0.5]
 [0.8 0.2]]
B:
[[0.8  0.2 ]
 [0.15 0.85]]
pi:
[0.5 0.5]
FIRST GUESSES:
[[0.15841746 0.84158254]
 [0.69941679 0.30058321]]
[[0.64650591 0.35349409]
 [0.02755445 0.97244555]]
[0.65474015 0.34525985]
log_likelyhood: -1767.3058386732198
final estimated values:
A:
[[0.32813015 0.67186985]
 [0.79134497 0.20865503]]
B:
[[0.86919994 0.13080006]
 [0.15539476 0.84460524]]
pi:
[0.98962917 0.01037083]
true values:
A:
[[0.5 0.5]
 [0.8 0.2]]
B:
[[0.8  0.2 ]
 [0.15 0.85]]
pi:
[0.5 0.5]
FIRST GUESSES:
[[0.33831001 0.66168999]
 [0.69200471 0.30799529]]
[[0.37771755 0.62228245]
 [0.23584274 0.76415726]]
[0.53240677 0.46759323]
log_likelyhood: -1612.462199597707
final estimated values:
A:
[[0.36289802 0.63710198]
 [0.71820439 0.28179561]]
B:
[[0.62744849 0.37255151]
 [0

C:\Users\tobia\AppData\Local\Temp\ipykernel_10260\3596776542.py:29: RuntimeWarning: invalid value encountered in scalar subtract
  loggamma[t, i] = logalpha[t, i] + logbeta[t, i] - logsumexp([
C:\Users\tobia\AppData\Local\Temp\ipykernel_10260\3596776542.py:47: RuntimeWarning: invalid value encountered in scalar subtract
  logalpha[t, i] + loga[i, j] + logb[j, observations[t + 1]] + logbeta[t + 1, j]


log_likelyhood: -inf
final estimated values:
FIRST GUESSES:
[[0.49897981 0.50102019]
 [1.         0.        ]]
[[0.34634471 0.65365529]
 [0.01       0.99      ]]
[0.31373776 0.68626224]
log_likelyhood: -1863.95050313956
final estimated values:
A:
[[6.37180816e-01 3.62819184e-01]
 [1.00000000e+00 1.00000000e-10]]
B:
[[0.72081341 0.27918659]
 [0.04716867 0.95283133]]
pi:
[0.96843062 0.03156938]
true values:
A:
[[0.5 0.5]
 [0.8 0.2]]
B:
[[0.8  0.2 ]
 [0.15 0.85]]
pi:
[0.5 0.5]
FIRST GUESSES:
[[0.48746063 0.51253937]
 [0.62857227 0.37142773]]
[[0.88024708 0.11975292]
 [0.14516463 0.85483537]]
[0.35875179 0.64124821]
log_likelyhood: -1364.7552995845485
final estimated values:
A:
[[0.46522837 0.53477163]
 [0.64035417 0.35964583]]
B:
[[0.87873297 0.12126703]
 [0.13755072 0.86244928]]
pi:
[0.81276294 0.18723706]
true values:
A:
[[0.5 0.5]
 [0.8 0.2]]
B:
[[0.8  0.2 ]
 [0.15 0.85]]
pi:
[0.5 0.5]
FIRST GUESSES:
[[0.60838738 0.39161262]
 [0.68199943 0.31800057]]
[[0.70907234 0.29092766]
 [0.01    

C:\Users\tobia\AppData\Local\Temp\ipykernel_10260\3814822106.py:78: RuntimeWarning: divide by zero encountered in log
  logpi = np.log(pi)


log_likelyhood: -1406.9164808692844
final estimated values:
A:
[[0.6846944  0.3153056 ]
 [0.86522441 0.13477559]]
B:
[[0.73342594 0.26657406]
 [0.01460413 0.98539587]]
pi:
[1. 0.]
true values:
A:
[[0.5 0.5]
 [0.8 0.2]]
B:
[[0.8  0.2 ]
 [0.15 0.85]]
pi:
[0.5 0.5]
FIRST GUESSES:
[[0.44879477 0.55120523]
 [1.         0.        ]]
[[0.94858952 0.05141048]
 [0.22899894 0.77100106]]
[0.57280244 0.42719756]
log_likelyhood: -1621.4684942345045
final estimated values:
A:
[[3.96576629e-01 6.03423371e-01]
 [1.00000000e+00 1.00000000e-10]]
B:
[[0.78147922 0.21852078]
 [0.14378557 0.85621443]]
pi:
[0.98476187 0.01523813]
true values:
A:
[[0.5 0.5]
 [0.8 0.2]]
B:
[[0.8  0.2 ]
 [0.15 0.85]]
pi:
[0.5 0.5]
FIRST GUESSES:
[[0.42843686 0.57156314]
 [0.90693774 0.09306226]]
[[0.75056575 0.24943425]
 [0.08106874 0.91893126]]
[0.57413534 0.42586466]
log_likelyhood: -1379.104149924594
final estimated values:
A:
[[0.47681624 0.52318376]
 [0.9089133  0.0910867 ]]
B:
[[0.78674462 0.21325538]
 [0.11534655 0.8846